In [6]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import (
    ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings)

In [7]:
loader = PyPDFDirectoryLoader("D:\Repos\GenAI\Data")
data = loader.load()

In [8]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001") #gemini-embedding-exp-03-07
chat_model = ChatGoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25")

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text_chunks = text_splitter.split_documents(data)
text_chunks[:2]

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-03-22T10:55:39+00:00', 'author': 'Shahan Shaik', 'moddate': '2025-03-22T10:55:39+00:00', 'source': 'D:\\Repos\\GenAI\\Data\\DVS_Gen_AI_Course_Content.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='DVS Technologies | Website: www.dvstechnologies.in | Mobile: 8892499499, 8892599599 \n \n \nDVS Generative AI Course Content \nFoundations of AI & Generative AI \nIntroduction to AI & Generative AI \nWhat is AI? Basics of Machine Learning & Deep Learning \nTraditional AI vs Generative AI \nKey applications across industries \nLarge Language Models (LLMs) Basics \nHow LLMs work: Transformers, Embeddings, Attention Mechanism \nPopular LLMs: GPT, LLaMA, Falcon, Mistral \nEthical AI & bias concerns \n Introduction to Prompt Engineering \nImportance of prompts in LLMs \nBest practices: Direct, Chain-of-Thought, Few-shot prompting \n \nPractical NLP & Text Processing \n

In [10]:
text_chunks[3].page_content

'AI in Healthcare \nMedical report summarization & clinical decision support \nAI in contract analysis & legal compliance \nHands-on: Use AI for extracting insights from legal/medical documents \n \nModel Deployment & Automation \nDeploying AI Models with FastAPI & Streamlit \nBuilding AI-powered web apps & APIs \nHands-on: Deploy a summarization or chatbot model using Streamlit \nAutomating Workflows with AI \nAI-powered email automation, scheduling, content creation \nHands-on: Automate report generation & email responses with AI \n \nCapstone Projects & Industry Applications \n Hands-on Real-World Projects (Will be customizable based on student needs) \nProject 1: AI-Powered Document Summarization (Extract insights from legal/financial docs) \nProject 2: Smart AI Chatbot for Customer Support (RAG-based FAQ bot) \nProject 3: AI Fraud Detection System (Analyzing transaction anomalies) \nProject Implementation & Refinement \nAdding UI/UX components using Streamlit'

In [11]:
len(text_chunks)

7

In [12]:
embedding_model.embed_query(text_chunks[0].page_content)

[0.014822223223745823,
 -0.04407105967402458,
 0.01721842586994171,
 0.025270063430070877,
 0.03716124966740608,
 0.006132985465228558,
 0.06797534972429276,
 0.021715348586440086,
 0.015849148854613304,
 0.03262944146990776,
 0.021238498389720917,
 -0.013007697649300098,
 -0.01958329789340496,
 0.000561158056370914,
 -0.008147829212248325,
 -0.007109756115823984,
 0.03957995027303696,
 -0.0027906436007469893,
 0.0007682883297093213,
 -0.037909384816884995,
 -0.0036968947388231754,
 -0.0034251913893967867,
 0.03808446601033211,
 -0.02884661965072155,
 -0.0007850597030483186,
 -0.0037277115043252707,
 0.0048371595330536366,
 -0.04201996698975563,
 -0.04315773770213127,
 0.03461689129471779,
 -0.04299754649400711,
 0.023462917655706406,
 -0.03402835130691528,
 0.0062956660985946655,
 0.010405456647276878,
 -0.012341124005615711,
 -0.0009359646937809885,
 0.018488965928554535,
 -0.0020991286728531122,
 0.007715420797467232,
 -0.0020535679068416357,
 -0.025699198246002197,
 0.0274942498654

## Config PineCone Client

In [13]:
# pip install Pinecone


In [14]:
from pinecone import Pinecone,ServerlessSpec
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

In [16]:
index_name = "example-index"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"multilingual-e5-large",
            "field_map":{"text": "chunk_text"}
        }
    )

In [17]:
index = pc.Index("example-index")

In [18]:
pc.list_indexes()

[
    {
        "name": "index1",
        "dimension": 1024,
        "metric": "cosine",
        "host": "index1-t1wqam0.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    },
    {
        "name": "example-index",
        "dimension": 1024,
        "metric": "cosine",
        "host": "example-index-t1wqam0.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    },
    {
        "name": "sample-movies",
        "dimension": 1024,
        "metric": "cosine",
        "host": "sample-movies-t

In [19]:
import time

index_name = "langchain-test-index2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [30]:
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4

vector_store = PineconeVectorStore(index=index_name, embedding=embedding_model)
uuids = [str(uuid4()) for _ in range(len(text_chunks))]
vector_store.add_documents(documents=text_chunks, ids=uuids)

AttributeError: 'str' object has no attribute 'upsert'

In [28]:
vector_store

In [22]:
len(text_chunks)

7

In [25]:
uuids[0]

'cb46fd48-670a-4e9a-a555-fd1b327a4bb8'

In [26]:
len(uuids)

7